# Project 3 - 3 Visualization

## imports

In [2]:
import math
import numpy as np
import pandas as pd
import random

import psycopg2

import json

import gmaps
import gmaps.geojson_geometries

from geographiclib.geodesic import Geodesic
import csv

## my_select_query_pandas() - function to run a select query and return rows in a Pandas dataframe

In [3]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)
    

## Connect to the Postgres database

In [4]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

## Create a cursor for the connection

In [5]:
cursor = connection.cursor()

## Connect to Google Maps using your api key;  edit the file gmap_api_key.txt and put in your api key

In [6]:
f = open('gmap_api_key.txt', 'r')
my_api_key = f.read()
f.close()

gmaps.configure(api_key=my_api_key)

## Basic map centered on Sather Gate at UC Berkeley; all point are (latitude, longitude) in decimal; zoom levels go from 1 to 21; 1 is world level; 21 is street level; default type of map is Road Map

In [7]:
sather_gate_berkeley = (37.870260430419115, -122.25950168579497)

# Beggining of Project 3 code 
## We want to map a cluster generated using the degree centrality algorithm 

## Load the CSV file with subgroup

We will color all the clusters in one color, and the central algorithm in a different color
We will load the CSV file which contains the results of the previous table


In [8]:
df = pd.read_csv('degree_centrality.csv')
df

,Unnamed: 0,unnamed,zip,n_lat,n_lon,customer_id,first_name,last_name,street,louvein_community,city,central_node
0,4,4,94005,37.682348,-122.406707,7158,Malissa,Latek,8346 Fieldstone Park,6860,Brisbane,0
1,9,9,94005,37.683216,-122.409611,7189,Kaja,Killiner,2122 Eagle Crest Plaza,6860,Brisbane,0
2,10,10,94005,37.682674,-122.414409,7188,Trever,Wais,13540 Manitowish Lane,6860,Brisbane,0
3,12,12,94005,37.683365,-122.407019,7186,Lucius,Gawne,6586 Dorton Alley,6860,Brisbane,0
4,17,17,94005,37.686030,-122.410622,7181,Fraser,Vasser,6313 Warbler Circle,6860,Brisbane,0
5,18,18,94005,37.687203,-122.409959,7180,Candie,Dayne,807 Roth Road,6860,Brisbane,0
6,25,25,94005,37.682012,-122.403878,7173,Joshuah,MacFadzan,135 Di Loreto Park,6860,Brisbane,0
7,26,26,94005,37.682657,-122.411316,7172,Francois,Farish,1 Texas Trail,6860,Brisbane,1
8,27,27,94005,37.681886,-122.416236,7171,Byrann,Feechum,5 Tony Plaza,6860,Brisbane,0
9,30,30,94005,37.682597,-122.398976,7168,Ashly,Lindup,47 Fieldstone Parkway,6860,Brisbane,0


## Loading subgroup of Louvain Community '6860'
### Overlaying the map with the Symbol Layers function: Black dots represent each customer, with the central node with an interactive marker

In [9]:
#color green is chosen for dollars sign $$$

#add degree central node with a marker

info_box_template = """
<dl>
<dt>Central_Node</dt><dd>{customer_id}</dd>
<dt>Latitude</dt><dd>{n_lat}</dd>
<dt>Longitude</dt><dd>{n_lon}</dd>
<dt>District</dt><dd>{louvein_community}</dd>

</dl>
"""

temp3_df = df.loc[df['central_node'] == 1]
central_node_markers = temp3_df[['n_lat','n_lon']] 


df_list_dict = temp3_df.to_dict('records')
df_list_dict
station_info = [info_box_template.format(**stations) for stations in df_list_dict]
marker_layer2 = gmaps.marker_layer(temp3_df[['n_lat','n_lon']], info_box_content=station_info)
#marker_layer2 = gmaps.marker_layer(central_node_markers)


central = (float(central_node_markers['n_lat']), float(central_node_markers['n_lon']))
fig = gmaps.figure(center=central, zoom_level=14)

df_markers = df[['n_lat','n_lon']]
marker_layer = gmaps.symbol_layer(df_markers, fill_color='black', stroke_color='black' )

fig.add_layer(marker_layer)
fig.add_layer(marker_layer2)



fig




Figure(layout=FigureLayout(height='420px'))